<a href="https://colab.research.google.com/github/Shashank-Shukla/PyTorch-practice/blob/master/PyTorch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

print("Torch version: ",torch.__version__)
print("Cuda active: ",torch.cuda.is_available())

Cuda active:  False


In [ ]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x


In [ ]:
class CNN(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    # for mnmist dataset, in_channel = 1 and num_classes = 10
    super(CNN, self).__init__()
    # same convolution: kernel=3x3 and padding=1x1 and stride=1x1
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
    # n_out = ((n_in + 2*pad - kernal)/stride) + 1
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.fc1 = nn.Linear(16*7*7, num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)
    return x
  

In [ ]:
model = CNN()
x = torch.randn(64,1,28,28)
print("X shape = ",x.shape)
print("Model shape: ",model(x).shape)


X shape =  torch.Size([64, 1, 28, 28])
Model shape:  torch.Size([64, 10])


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
in_channel = 1
num_classes = 10
lr = 0.001
bs = 64
epochs = 20

# Loading data train and test respectively
train_dataset = datasets.MNIST(root='dataset/',train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/',train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=True)

# Initialize model
model = CNN().to(device)

# Setting loss and optimizer criterion
criterion_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training the network
for epoch in range(epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    # Run in cuda
    data = data.to(device)
    targets = targets.to(device)

    # Forward propagation
    scores = model(data)
    loss = criterion_loss(scores, targets)

    # Backward propagation
    optimizer.zero_grad()
    loss.backward()

    # Gradient descent
    optimizer.step()


In [15]:
def check_acc(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training-set")
  else:
    print("Checking accuracy on testing-set")
  num_correct = 0
  num_sample = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x = x.to(device)

      scores = model(x)
      _, preds = scores.max(1)
      num_correct += (preds == y).sum()
      num_sample += preds.size(0)

    print(f"Got {num_correct}/{num_sample} = {float(num_correct)/num_sample * 100:.2f}")

  model.train()


In [16]:
check_acc(train_loader, model)
check_acc(test_loader, model)

Checking accuracy on training-set
Got 59790/60000 = 99.65
Checking accuracy on testing-set
Got 9883/10000 = 98.83
